In [9]:
import sys
import os
import logging
 
# Add the project root directory to Python path
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from src.playstore_scraper import PlayStoreScraper
from src.review_preprocessor import ReviewPreprocessor
import pandas as pd
import logging
%reload_ext autoreload
%autoreload 2 


In [10]:
scraper = PlayStoreScraper()
reviews_by_bank = scraper.scrape_all()

        
 

2025-06-09 01:34:57,102 - INFO - Scraping reviews for cbe
2025-06-09 01:34:58,645 - INFO - Successfully fetched 600 reviews for com.combanketh.mobilebanking
2025-06-09 01:34:58,662 - ERROR - Error processing cbe: Cannot save file into a non-existent directory: '..\data\raw'
2025-06-09 01:34:58,666 - INFO - Scraping reviews for boe
2025-06-09 01:34:59,852 - INFO - Successfully fetched 600 reviews for com.boa.boaMobileBanking
2025-06-09 01:34:59,861 - ERROR - Error processing boe: Cannot save file into a non-existent directory: '..\data\raw'
2025-06-09 01:34:59,862 - INFO - Scraping reviews for dashen
2025-06-09 01:35:01,029 - INFO - Successfully fetched 450 reviews for com.dashen.dashensuperapp
2025-06-09 01:35:01,036 - ERROR - Error processing dashen: Cannot save file into a non-existent directory: '..\data\raw'


In [8]:
reviews_by_bank

{}

In [3]:
 # Initialize and run preprocessor
preprocessor = ReviewPreprocessor()


In [4]:
preprocessor.process_all_banks()